In [1]:
import pandas as pd
from gensim.models import Word2Vec
from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from pep_modules import patientai as pai

In [3]:
rx = pd.read_csv(r'readmissions_data/PRESCRIPTIONS.csv')

C:\Users\mg21929\AppData\Local\Continuum\miniconda3\envs\tfgpu\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
rx['STARTDATE'] = pd.to_datetime(rx['STARTDATE'])

In [5]:
rx.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [6]:
rx.DRUG.nunique()

4525

In [7]:
rx.isna().sum()

ROW_ID                     0
SUBJECT_ID                 0
HADM_ID                    0
ICUSTAY_ID           1447708
STARTDATE               3182
ENDDATE                 5421
DRUG_TYPE                  0
DRUG                       0
DRUG_NAME_POE        1664234
DRUG_NAME_GENERIC    1662989
FORMULARY_DRUG_CD       1933
GSN                   507164
NDC                     4463
PROD_STRENGTH           1362
DOSE_VAL_RX             1350
DOSE_UNIT_RX            1342
FORM_VAL_DISP           1355
FORM_UNIT_DISP          1409
ROUTE                   1156
dtype: int64

In [8]:
rx.dtypes

ROW_ID                        int64
SUBJECT_ID                    int64
HADM_ID                       int64
ICUSTAY_ID                  float64
STARTDATE            datetime64[ns]
ENDDATE                      object
DRUG_TYPE                    object
DRUG                         object
DRUG_NAME_POE                object
DRUG_NAME_GENERIC            object
FORMULARY_DRUG_CD            object
GSN                          object
NDC                         float64
PROD_STRENGTH                object
DOSE_VAL_RX                  object
DOSE_UNIT_RX                 object
FORM_VAL_DISP                object
FORM_UNIT_DISP               object
ROUTE                        object
dtype: object

In [9]:
px = pd.read_csv(r'readmissions_data/PROCEDURES_ICD.csv')

In [10]:
px.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [11]:
dx =  pd.read_csv(r'readmissions_data/DIAGNOSES_ICD.csv')

In [12]:
dx.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [113]:
target_30 = pd.read_pickle('target_30.pkl')
target_30.head()

,pat_id,switch_flag,HADM_ID_lst
0,2,0,[163353]
1,3,0,[145834]
2,4,0,[185777]
3,5,0,[178980]
4,6,0,[107064]


In [105]:
target_30.pat_id.nunique(),target_30[target_30.switch_flag!=2].pat_id.nunique()

(46520, 45980)

In [114]:
a = target_30[target_30.pat_id ==2]['HADM_ID_lst'].tolist()[0]

In [174]:
def grpby_filter_journey_px_dx(df, target):    
    if df['SUBJECT_ID'].tolist()[0] in target[target['switch_flag']==1]['pat_id'].tolist():
        lst = target[target.pat_id==df['SUBJECT_ID'].tolist()[0]]['HADM_ID_lst'].tolist()[0]
        tmp = df[df['HADM_ID'].isin(lst)]
        tmp.sort_values(['SUBJECT_ID','HADM_ID','SEQ_NUM'], ascending = [True, False, False], inplace = True)
        time_stamp = list(range(1,len(tmp)+1))
        tmp['seq_num_modified'] = time_stamp
    elif df['SUBJECT_ID'].tolist()[0] in target[target['switch_flag']==0]['pat_id'].tolist():
        tmp = df
        tmp.sort_values(['SUBJECT_ID','HADM_ID','SEQ_NUM'], ascending = [True, False, False], inplace = True)
        time_stamp = list(range(1,len(tmp)+1))
        tmp['seq_num_modified'] = time_stamp
    else:
        tmp = pd.Series([])
    return tmp

def grpby_filter_journey_rx(df, target):    
    if df['SUBJECT_ID'].tolist()[0] in target[target['switch_flag']==1]['pat_id'].tolist():
        lst = target[target.pat_id==df['SUBJECT_ID'].tolist()[0]]['HADM_ID_lst'].tolist()[0]
        tmp = df[df['HADM_ID'].isin(lst)]
        tmp.sort_values(['SUBJECT_ID','HADM_ID','STARTDATE'], ascending = [True, False, False], inplace = True)
        time_stamp = list(range(1,len(tmp)+1))
        tmp['seq_num_modified'] = time_stamp
    elif df['SUBJECT_ID'].tolist()[0] in target[target['switch_flag']==0]['pat_id'].tolist():
        tmp = df
        tmp.sort_values(['SUBJECT_ID','HADM_ID','STARTDATE'], ascending = [True, False, False], inplace = True)
        time_stamp = list(range(1,len(tmp)+1))
        tmp['seq_num_modified'] = time_stamp
    else:
        tmp = pd.Series([])
    return tmp

In [117]:
px_mod = px.groupby('SUBJECT_ID').apply(grpby_filter_journey_px_dx, target = target_30)
px_mod = px_mod.drop(columns = ['SUBJECT_ID']).reset_index().drop(columns=['level_1'])
px_mod.drop(columns=[0],inplace = True)

36 2


C:\Users\mg21929\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app


109 2
222 2
402 2
518 2
580 2
634 2
711 2
808 2
852 2
952 2
975 2
1076 2
1339 2
1478 2
1620 2
1709 2
1931 2
1988 2
2040 2
2090 2
2136 2
2187 2
2264 2
2265 2
2338 2
2365 2
2378 2
2561 2
2589 2
2598 2
2639 2
2921 2
3078 2
3084 2
3181 2
3386 2
3393 2
3482 2
3497 2
3506 2
3543 2
3575 2
3600 2
3748 2
3830 2
3866 2
3868 2
3917 2
3935 2
3952 2
3969 2
3977 2
4113 2
4137 2
4316 2
4329 2
4655 2
4787 2
4916 2
5008 2
5058 2
5060 2
5239 2
5343 2
5397 2
5406 2
5689 2
5691 2
5727 2
5771 2
5841 2
6365 2
6428 2
6440 2
6451 2
6718 2
6824 2
6828 2
6953 2
7095 2
7105 2
7107 2
7138 2
7192 2
7275 2
7326 2
7371 2
7478 2
7546 2
7681 2
7726 2
7787 2
7809 2
7908 2
8060 2
8344 2
8471 2
8492 2
8556 2
8698 2
8799 2
8896 2
8947 2
9186 2
9253 2
9256 2
9266 2
9339 2
9402 2
9545 2
9686 2
9725 2
9736 2
9782 2
9828 2
9882 2
9973 2
10088 2
10144 2
10160 2
10188 2
10254 2
10434 2
10487 2
10502 2
10515 2
10531 2
10635 2
10642 2
10679 2
10721 2
10751 2
10774 2
10814 2
10950 2
10973 2
11003 2
11216 2
11236 2
11318 2
11342 2


In [192]:
px_mod['type'] = 'px'

In [130]:
# Patients removed
px.SUBJECT_ID.nunique()- px_mod.SUBJECT_ID.nunique()

683

In [165]:
px_mod.head(10)

,SUBJECT_ID,HADM_ID,ICD9_CODE,ROW_ID,SEQ_NUM,seq_num_modified
0,2,163353.0,9955.0,109376.0,1.0,1.0
1,3,145834.0,966.0,44189.0,6.0,1.0
2,3,145834.0,3893.0,44188.0,5.0,2.0
3,3,145834.0,9672.0,44187.0,4.0,3.0
4,3,145834.0,8964.0,44186.0,3.0,4.0
5,3,145834.0,9962.0,44185.0,2.0,5.0
6,3,145834.0,9604.0,44184.0,1.0,6.0
7,4,185777.0,3323.0,74268.0,3.0,1.0
8,4,185777.0,8872.0,74267.0,2.0,2.0
9,4,185777.0,3893.0,74266.0,1.0,3.0


In [146]:
dx_mod = dx.groupby('SUBJECT_ID').apply(grpby_filter_journey_px_dx, target = target_30)
dx_mod = dx_mod.drop(columns = ['SUBJECT_ID']).reset_index().drop(columns=['level_1',0])

C:\Users\mg21929\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [191]:
dx_mod['type'] = 'dx'

In [164]:
dx_mod[dx_mod.SUBJECT_ID==17]

,SUBJECT_ID,HADM_ID,ICD9_CODE,ROW_ID,SEQ_NUM,seq_num_modified
72,17,194023.0,2724,84.0,4.0,1.0
73,17,194023.0,V1259,83.0,3.0,2.0
74,17,194023.0,45829,82.0,2.0,3.0
75,17,194023.0,7455,81.0,1.0,4.0
76,17,161087.0,2724,80.0,8.0,5.0
77,17,161087.0,71946,79.0,7.0,6.0
78,17,161087.0,7220,78.0,6.0,7.0
79,17,161087.0,311,77.0,5.0,8.0
80,17,161087.0,4589,76.0,4.0,9.0
81,17,161087.0,78551,75.0,3.0,10.0


In [179]:
tqdm.pandas()

rx_mod = rx.groupby('SUBJECT_ID').progress_apply(grpby_filter_journey_rx, target = target_30)
rx_mod = rx_mod.drop(columns = ['SUBJECT_ID']).reset_index().drop(columns=['level_1',0])

C:\Users\mg21929\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [187]:
rx_mod.head(50)

,SUBJECT_ID,DOSE_UNIT_RX,DOSE_VAL_RX,DRUG,DRUG_NAME_GENERIC,DRUG_NAME_POE,DRUG_TYPE,ENDDATE,FORMULARY_DRUG_CD,FORM_UNIT_DISP,...,GSN,HADM_ID,ICUSTAY_ID,NDC,PROD_STRENGTH,ROUTE,ROW_ID,STARTDATE,seq_num_modified,ICD9_CODE
0,2,mg,15.5,NEO*IV*Gentamicin,NaN,NaN,MAIN,2138-07-20 00:00:00,GENT10I,VIAL,...,009298,163353.0,243653.0,6.332302e+10,10mg/mL-2mL,IV,2968761.0,2138-07-18,1.0,NEO*IV*Gentamicin
1,2,ml,2.4,Syringe (Neonatal) *D5W*,NaN,NaN,BASE,2138-07-20 00:00:00,NEOSYRD5W,ml,...,NaN,163353.0,243653.0,0.000000e+00,1 Syringe,IV,2968759.0,2138-07-18,2.0,Syringe (Neonatal) *D5W*
2,2,mg,500,Ampicillin Sodium,NaN,NaN,MAIN,2138-07-21 00:00:00,AMP500I,VIAL,...,008937,163353.0,243653.0,6.332304e+10,500mg Vial,IV,2968762.0,2138-07-18,3.0,Ampicillin Sodium
3,2,VIAL,1,Send 500mg Vial,NaN,NaN,BASE,2138-07-21 00:00:00,AMPVL,VIAL,...,NaN,163353.0,243653.0,0.000000e+00,Send 500mg Vial,IV,2968760.0,2138-07-18,4.0,Send 500mg Vial
4,4,ml,250,D5W,NaN,NaN,BASE,2191-03-23 00:00:00,D5W250,ml,...,001972,185777.0,NaN,3.380017e+08,250mL Bag,IV,1213115.0,2191-03-23,1.0,D5W
5,4,mg,750,Vancomycin HCl,NaN,NaN,MAIN,2191-03-23 00:00:00,VANC500I,VIAL,...,009331,185777.0,NaN,6.332302e+10,500mg Vial,IV,1213121.0,2191-03-23,2.0,Vancomycin HCl
6,4,mEq,40,Potassium Chloride,Potassium Chloride,Potassium Chloride,MAIN,2191-03-23 00:00:00,MICROK10,CAP,...,001248,185777.0,NaN,5.817700e+10,10mEq ER,PO,1213855.0,2191-03-22,3.0,Potassium Chloride
7,4,UNIT,8,Insulin,Insulin Human NPH,Insulin,MAIN,2191-03-23 00:00:00,INNHI,ml,...,001740,185777.0,NaN,2.831501e+06,10mL Vial,SC,1213856.0,2191-03-22,4.0,Insulin
8,4,UNIT,0,Insulin,Insulin,Insulin,MAIN,2191-03-23 00:00:00,INSULIN,VIAL,...,001723,185777.0,NaN,0.000000e+00,Dummy Pkg for POE pump and PHA SS ordering,SC,1213857.0,2191-03-22,5.0,Insulin
9,4,UNIT,12,Lantus,Insulin Glargine,Lantus,MAIN,2191-03-23 00:00:00,GLAR100I,VIAL,...,047780,185777.0,NaN,8.822203e+07,10mLVial,SC,1213860.0,2191-03-22,6.0,Lantus


In [190]:
rx_mod['ICD9_CODE'] = rx_mod['DRUG']
rx_mod['type'] = 'rx'

In [193]:
df_combined = pd.concat([rx_mod[['SUBJECT_ID','ICD9_CODE','seq_num_modified','type']]
                         , px_mod[['SUBJECT_ID','ICD9_CODE','seq_num_modified','type']]
                        , dx_mod[['SUBJECT_ID','ICD9_CODE','seq_num_modified','type']]])

In [230]:
df_combined.head()

,SUBJECT_ID,ICD9_CODE,seq_num_modified,type
0,2,NEO*IV*Gentamicin,1.0,rx
1,2,Syringe (Neonatal) *D5W*,2.0,rx
2,2,Ampicillin Sodium,3.0,rx
3,2,Send 500mg Vial,4.0,rx
4,4,D5W,1.0,rx


In [199]:
def grp_fun(df, a = 'rx', b = 'px', c = 'dx'):
    tmp1 = df[df.type == a].sort_values('seq_num_modified', ignore_index = True)
    tmp2 = df[df.type == b].sort_values('seq_num_modified', ignore_index = True)
    tmp3 = df[df.type == c].sort_values('seq_num_modified', ignore_index = True)
    tmp = pd.concat([tmp1, tmp2, tmp3], ignore_index=True)
    time_stamp = list(range(1,len(tmp)+1))
    tmp['seq_num_final'] = time_stamp
    return tmp

In [202]:
df_new_seq = df_combined.groupby('SUBJECT_ID').progress_apply(grp_fun, a = 'rx', b = 'px', c = 'dx').reset_index(drop=True)

In [213]:
df_new_seq.rename(columns = {'SUBJECT_ID':'pat_id'}, inplace = True)

In [214]:
df_new_seq.head()

,pat_id,ICD9_CODE,seq_num_modified,type,seq_num_final,cohort
0,2,NEO*IV*Gentamicin,1.0,rx,1,c1
1,2,Syringe (Neonatal) *D5W*,2.0,rx,2,c1
2,2,Ampicillin Sodium,3.0,rx,3,c1
3,2,Send 500mg Vial,4.0,rx,4,c1
4,2,9955,1.0,px,5,c1


In [211]:
df_new_seq.shape

(4622362, 6)

In [205]:
target = target_30[target_30.switch_flag!=2]

In [222]:
target.shape, df_new_seq.pat_id.nunique()

((45980, 3), 45980)

In [218]:
df_new_seq = pd.merge(df_new_seq,target,on='pat_id',how='left')

In [ ]:
df_new_seq['cohort'] = 'c1'

In [229]:
df_new_seq.head()

,pat_id,ICD9_CODE,seq_num_modified,type,seq_num_final,cohort,switch_flag,HADM_ID_lst
0,2,NEO*IV*Gentamicin,1.0,rx,1,c1,0,[163353]
1,2,Syringe (Neonatal) *D5W*,2.0,rx,2,c1,0,[163353]
2,2,Ampicillin Sodium,3.0,rx,3,c1,0,[163353]
3,2,Send 500mg Vial,4.0,rx,4,c1,0,[163353]
4,2,9955.0,1.0,px,5,c1,0,[163353]


In [231]:
df_new_seq.to_pickle('pat_journey_rx_px_dx.pkl')